In [1]:
import os
os.environ["PYTHON_EXEC"] = "/Users/jessieyu/.pyenv/versions/qka-demo/bin/python3"
os.environ["NTC_DOC_FILE"] = "runtime/qka_doc.json"
os.environ["NTC_PROGRAM_FILE"] = "runtime/qka_runtime_outer.py"

In [2]:
from qiskit import IBMQ

In [3]:
provider = IBMQ.load_account()

In [4]:
provider.runtime.programs()

Runtime Program QKA:
  Description: Quantum kernel alignment algorithm.
  Parameters:
    feature_map:
      description: quantum feature map
      type: FeatureMapQuantumControl
    data:
      description: NxD array, where N is the number of data points, D is the feature dimension.
      type: numpy.ndarray
    labels:
      description: Nx1 array of +/-1, where N is the number of data points
      type: numpy.ndarray
    lambda_plus:
      description: Plus kernel parameter
      type: numpy.ndarray
    lambda_minus:
      description: Minus kernel parameter
      type: numpy.ndarray
    C:
      description: penalty parameter for soft-margin
      type: float
  Returns:
    cost_plus:
      description: SVM objective function evaluated at (alpha_+, lambda_+)
      type: float
    cost_minus:
      description: SVM objective function evaluated at (alpha_-, lambda_-)
      type: float


In [5]:
import numpy as np
from qka.featuremaps import FeatureMapForrelation

num_samples=5  # number of samples per class in the input data
num_features=2 # number of features in the input data
depth=2        # depth of the feature map circuit
C=1            # SVM soft-margin penalty
spsa_steps=2   # number of SPSA iterations

# Define the feature map and its initial parameters:
fm = FeatureMapForrelation(feature_dimension=num_features, depth=depth, entangler_map=None)
lambda_initial = np.random.uniform(-1,1, size=(fm._num_parameters))

# create random test data and labels:
x_train = np.random.rand(2*num_samples, num_features)
y_train = np.concatenate((-1*np.ones(num_samples), np.ones(num_samples)))

backend = provider.backend.ibmq_qasm_simulator

In [6]:
def interim_result_callback(interim_result):
    print(f"interim result: {interim_result}\n")

In [7]:
# Call QKA runtime program.
runtime_params = {
    'feature_map': fm.to_dict(),
    'data': x_train,
    'labels': y_train,
    'lambda_initial': lambda_initial,
    'spsa_steps': 2,
    'C': C
}
result = provider.runtime.run(program_name="QKA",
                              backend=backend,
                              params=runtime_params,
                              callback=interim_result_callback,
                             ).result()
print("final result:")
print(f"aligned_kernel_parameters: {result['aligned_kernel_parameters']}")
print(f"aligned_kernel_matrix: {result['aligned_kernel_matrix']}")

interim result: {'cost': 7.573571427178223, 'lambda': array([ 0.54118207,  0.15037056,  0.60169902, -0.57977013]), 'cost_plus': 7.6806636414656175, 'cost_minus': 7.466479212890828, 'cost_final': 7.573571427178223}

interim result: {'cost': 8.067868423040537, 'lambda': array([ 0.53141793,  0.14060643,  0.61146315, -0.57000599]), 'cost_plus': 8.206049892959566, 'cost_minus': 7.929686953121507, 'cost_final': 8.067868423040537}

final result:
best_kernel_parameters: [ 0.10726     0.0290977   0.12131622 -0.11497761]
best_kernel_matrix: [[1.         0.00292969 0.00195312 0.13476562 0.06152344 0.46386719
  0.015625   0.38476562 0.06933594 0.109375  ]
 [0.00292969 1.         0.95214844 0.40722656 0.03417969 0.40136719
  0.95898438 0.49511719 0.06640625 0.671875  ]
 [0.00195312 0.95214844 1.         0.34179688 0.03417969 0.44921875
  0.90039062 0.52832031 0.04980469 0.79785156]
 [0.13476562 0.40722656 0.34179688 1.         0.33691406 0.37402344
  0.37890625 0.39941406 0.06640625 0.1640625 ]
 [0